In [1]:
import os

In [2]:
%pwd

'c:\\Users\\herma\\OneDrive\\Documents\\End-to-end-ML-with-mlflow\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\herma\\OneDrive\\Documents\\End-to-end-ML-with-mlflow'

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')

In [6]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict 

In [11]:
from mlProject import logger
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [ ]:
def get_data_validation_config(self) -> DataValidationConfig:
    config = self.config.data_ingestion
    schema = self.schema.COLUMNS

    create_directories([config.root_dir])

    data_validation_config = DataValidationConfig(
        root_dir=config.root_dir,
        STATUS_FILE=config.STATUS_FILE,
        unzip_data_dir=config.unzip_data_dir,
        all_schema=schema
    )
    return data_validation_config

In [12]:
import os
import pandas as pd
from mlProject import logger

In [21]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )
        return data_validation_config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            # validation_status = set(all_cols).issubset(all_schema)
            validation_status = set(all_cols) == set(all_schema)
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"validation status: {validation_status}")

            # for col in all_cols:
            #     if col not in all_schema:
            #         validation_status = False
            #     else:
            #         validation_status = False
            #     with open(self.config.STATUS_FILE, 'w') as f:
            #         f.write(f"validation status: {validation_status}")
            return validation_status
        except Exception as e:
            logger.exception("all columns in data are not the same from schema yaml file")
            raise e

In [23]:
from mlProject.config.configuration import ConfigurationManager

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-01-12 11:40:59,967: INFO: common:  yaml file: config\config.yaml loaded successfully]
[2024-01-12 11:40:59,968: INFO: common:  yaml file: params.yaml loaded successfully]
[2024-01-12 11:40:59,980: INFO: common:  yaml file: schema.yaml loaded successfully]
[2024-01-12 11:40:59,982: INFO: common:  Create directory at: artifacts]
[2024-01-12 11:40:59,983: INFO: configuration:  self.config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}}]
[2024-01-12 11:40:59,984: INFO: common:  Create directory at: artifacts/data_validation]
[2024-01-12 11:40:59,990: INFO: 735691400:  se